In [1]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager

#hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory


# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net
from source.custom_cnn.dropout_batch_norm_cnn import DBN_cnn
from source.pre_trained.vgg16 import PretrainedVGG16_cnn
from source.pre_trained.resnet_cnn import PretrainedRESNET_cnn



In [3]:

gpus_per_trial = 2
max_num_epochs = 40
num_samples = 10
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
local_dir = ConfigManager.get_tuning_results_path()
    
config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.Adam]),
    "lr":0.001,
    "weight_decay": 0.0005
    
}

config_net_1 = {
    "type": tune.choice([PretrainedVGG16_cnn]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1])}


config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_kaggle())
    ]),
    "batch_size": 64,
    "epochs": 40,
    "net": tune.choice([config_net_1]),
    "tuning_id": "XYZ"
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20, overwrite=True, max_report_frequency=10, max_error_rows=5)

In [4]:
tuner = HyperparameteresTunner()

In [5]:
best_model = tuner.tune(dataset_name, local_dir, config, scheduler, reporter, num_samples=1)

Trial name,status,loc,data_loaders_factory,net,net/criterion,net/criterion/type,net/optimizer,net/optimizer/type,net/type
__train_validate_5b8f2_00000,PENDING,,"Train_transformer:Compose( Resize(size=(244, 244), interpolation=bilinear, max_size=None, antialias=None) RandomHorizontalFlip(p=0.5) RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0) RandomAffine(degrees=[0.0, 0.0], scale=(0.8, 1.2), shear=[-10.0, 10.0]) ColorJitter(brightness=[0.8, 1.2], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=None) ToTensor() Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) ); Test_transformer:Compose( Resize(size=(244, 244), interpolation=bilinear, max_size=None, antialias=None) ToTensor() Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) )","{'type': <class 'source.pre_trained.vgg16.PretrainedVGG16_cnn'>, 'optimizer': {'type': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'weight_decay': 0.0005}, 'criterion': {'type': <class 'torch.nn.modules.loss.CrossEntropyLoss'>}}",{'type': <class 'torch.nn.modules.loss.CrossEntropyLoss'>},<class 'torch.nn.modules.loss.CrossEntropyLoss'>,"{'type': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'weight_decay': 0.0005}",<class 'torch.optim.adam.Adam'>,<class 'source.pre_trained.vgg16.PretrainedVGG16_cnn'>


2022-04-03 17:16:04,293	ERROR tune.py:635 -- Trials did not complete: [__train_validate_5b8f2_00000]
2022-04-03 17:16:04,293	INFO tune.py:639 -- Total run time: 266.70 seconds (266.41 seconds for the tuning loop).
2022-04-03 17:16:04,294	WARNING tune.py:643 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
2022-04-03 17:16:04,299	WARNING experiment_analysis.py:533 -- Could not find best trial. Did you pass the correct `metric` parameter?


AttributeError: 'NoneType' object has no attribute 'config'